In [1]:
import json
import altair as alt
from altair import expr, datum
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import requests

In [2]:
import colorsys
from matplotlib.colors import to_hex, to_rgb


def scale_lightness(rgb, scale_l):
    rgbhex = False
    if "#" in rgb:
        rgb = to_rgb(rgb)
        rgbhex = True
    # convert rgb to hls
    h, l, s = colorsys.rgb_to_hls(*rgb)
    # manipulate h, l, s values and return as rgb
    c = colorsys.hls_to_rgb(h, min(1, l * scale_l), s=s)
    if rgbhex:
        c = to_hex(c)
    return c

In [4]:
LOCAL = True

if LOCAL:
    local_suffix = "_local"
else:
    local_suffix = ""

In [5]:
%%capture pwd
!pwd

In [6]:
# uid = "2021-05-05-which-firms-and-industries-have-been-most-affected-by-covid-update"  # article unique ID
uid = pwd.stdout.split("/")[-1].split("\r")[0]
eco_git_home = (
    "https://raw.githubusercontent.com/EconomicsObservatory/ECOvisualisations/main/"
)
eco_git_path = eco_git_home + "articles/" + uid + "/data/"
vega_embed = requests.get(eco_git_home + "guidelines/html/vega-embed.html").text
colors = json.loads(
    requests.get(eco_git_home + "guidelines/colors/eco-colors.json").content
)
category_color = json.loads(
    requests.get(eco_git_home + "guidelines/colors/eco-category-color.json").content
)
hue_color = json.loads(
    requests.get(eco_git_home + "guidelines/colors/eco-single-hue-color.json").content
)
mhue_color = json.loads(
    requests.get(eco_git_home + "guidelines/colors/eco-multi-hue-color.json").content
)
div_color = json.loads(
    requests.get(eco_git_home + "guidelines/colors/eco-diverging-color.json").content
)
config = json.loads(
    requests.get(eco_git_home + "guidelines/charts/eco-global-config.json").content
)
height = config["height"]
width = config["width"]
uid, height, width

('2021-07-21-will-people-go-back-to-the-office-after-the-pandemic', 300, 500)

# Fig 1

In [48]:
df = pd.read_excel("raw/figures.xlsx",sheet_name='Fig1',skiprows=6).dropna().replace('-',0)
df=df.set_index('Unnamed: 0').stack().reset_index()
df.columns=['income','category','value']
df['value']/=100

In [49]:
f = "fig1_income"
f1 = eco_git_path + f + ".csv"
df.to_csv("data/" + f + ".csv")
f += local_suffix
open("visualisation/" + f + ".html", "w").write(
    vega_embed.replace(
        "JSON_PATH", f1.replace("/data/", "/visualisation/").replace(".csv", ".json")
    )
)
if LOCAL:
    f1 = df
readme="### "+f+'\n!["'+f+'"](visualisation/'+f+'.png "'+f+'")\n\n'
df.head()

,income,category,value
0,£50000 or more,I will work all my hours from my usual place o...,0.05
1,£50000 or more,Hybrid working,0.93
2,£50000 or more,I will continue to work from home and not retu...,0.01
3,£50000 or more,Other,0.01
4,£40000 up to £50000,I will work all my hours from my usual place o...,0.07


In [58]:
base = (
    alt.Chart(f1)
    .encode(tooltip="tooltip:N")
    .transform_calculate(tooltip="datum.income+' 🏢 '+datum.category+' 📈 '+round(datum.value*100)+'%'")
).transform_calculate(x="round(datum.value*100)")
bars1 = base.mark_bar(height=15, opacity=0.9, color=colors["eco-turquiose"]).encode(
    x=alt.X(
        "value:Q",
        stack="zero",
        axis=alt.Axis(
            grid=False,
            title="",
            labelColor=colors["eco-gray"],
            titleColor=colors["eco-gray"],
            tickColor=colors["eco-gray"],
            domainColor=colors["eco-gray"],
            format=".0%",
            tickCount=10,
            orient="bottom",
            labelAngle=0,
        ),
        scale=alt.Scale(
            domain=[0,1]
        )
    ),
    order="income:N",
    y=alt.Y(
        "income:N",
        sort=[],
        axis=alt.Axis(
            grid=False,
            title="% by income band",
            titleX=0,
            titleY=-5,
            titleBaseline="bottom",
            titleAngle=0,
            titleAlign="left",
            labelColor=colors["eco-gray"],
            titleColor=colors["eco-gray"],
            tickColor=colors["eco-gray"],
            domainColor=colors["eco-gray"],
        ),
    ),
    color=alt.Color(
        "category:N",
        legend=alt.Legend(title="",labelLimit=400),
        sort=[],
        scale=alt.Scale(
            range=[colors["eco-mid-blue"],colors["eco-turquiose"],
                    colors["eco-red"], colors["eco-gray"]],
        ),
    ),
)
text1 = bars1.mark_text(dx=-2, dy=1, align="right").encode(
    text=alt.Text("x:Q", format=".0f"),
    x=alt.X("value:Q", stack="zero"),
    order="income:N",
    color=alt.condition(
        datum.value < 0.04,
        alt.ColorValue(None),
        alt.ColorValue("white"),
    ),
)
layer1 = (
    ((bars1 + text1).properties(height=alt.Step(20), width=200))
    .configure_view(stroke=None)
    .properties(title="")
)
layer1.save("visualisation/" + f + "a.json")
layer1.save("visualisation/" + f + ".png")
open("README.md",'w').write(readme)
layer1

WARN Channel order is inappropriate for nominal field, which has no inherent order.
WARN Channel order is inappropriate for nominal field, which has no inherent order.
Fontconfig error: Cannot load default config file


alt.LayerChart(...)

# Fig 2

In [85]:
df = pd.read_excel("raw/figures.xlsx",sheet_name='Fig2_Fig3',skiprows=2,usecols='B:F',nrows=10)
df=df.set_index(df.columns[0]).stack().reset_index()
df.columns=['advantage','age','value']
df['value']/=100
df=df[::-1]

In [86]:
f = "fig2_advantages"
f2 = eco_git_path + f + ".csv"
df.to_csv("data/" + f + ".csv")
f += local_suffix
open("visualisation/" + f + ".html", "w").write(
    vega_embed.replace(
        "JSON_PATH", f2.replace("/data/", "/visualisation/").replace(".csv", ".json")
    )
)
if LOCAL:
    f2 = df
readme="### "+f+'\n!["'+f+'"](visualisation/'+f+'.png "'+f+'")\n\n'
df.head()

,advantage,age,value
39,Improved work-life balance,50-69,0.64
38,Improved work-life balance,30-49,0.67
37,Improved work-life balance,16-29,0.62
36,Improved work-life balance,Overall,0.65
35,Fewer distractions,50-69,0.48


In [110]:
bars = alt.Chart(f2).encode(
    x=alt.X(
        "value:Q",
        stack=False,
        title="",
        axis=alt.Axis(
            grid=False,
            title="",
            titleAnchor="end",
            titleX=415,
            titleY=7,
            format='.0%',
            labelColor=colors["eco-gray"],
            titleColor=colors["eco-gray"],
            tickColor=colors["eco-gray"],
            domainColor=colors["eco-gray"],
        ),
    ),
    y=alt.Y("advantage:N", title="", axis=None, sort=[]),
    tooltip="tooltip:N"
).transform_calculate(tooltip="datum.advantage+' 🏢 '+datum.age+' 📈 '+round(datum.value*100)+'%'")

bars1 = bars.mark_bar(
    size=11, yOffset=-14, color=colors["eco-turquiose"], opacity=0.8
).transform_filter("datum.age=='16-29'")
bars2 = bars.mark_bar(
    size=11, yOffset=-26, color=colors["eco-green"], opacity=0.8
).transform_filter("datum.age=='Overall'")
bars3 = bars.mark_bar(
    size=11, yOffset=-2, color=colors["eco-light-blue"], opacity=0.8
).transform_filter("datum.age=='30-49'")
bars4 = bars.mark_bar(
    size=11, yOffset=10, color=colors["eco-mid-blue"], opacity=0.8
).transform_filter("datum.age=='50-69'")
labels1 = bars1.mark_text(
    align="left",
    xOffset=5,
    yOffset=-14,
    color=colors["eco-turquiose"],
).encode(text=alt.Text("value:N",format='.0%'))
labels2 = bars2.mark_text(
    align="left",
    xOffset=5,
    yOffset=-26,
    color=colors["eco-green"],
).encode(text=alt.Text("value:N",format='.0%'))
labels3 = bars3.mark_text(
    align="left",
    xOffset=5,
    yOffset=-2,
    color=colors["eco-light-blue"],
).encode(text=alt.Text("value:N",format='.0%'))
labels4 = bars4.mark_text(
    align="left",
    xOffset=5,
    yOffset=10,
    color=colors["eco-mid-blue"],
).encode(text=alt.Text("value:N",format='.0%'))
titles = (
    bars.mark_text(
        align="left", size=11, baseline="bottom", yOffset=-35, color=colors["eco-gray"]
    )
    .encode(text="advantage:N", x=alt.X("x0:Q",axis=alt.Axis(format='.0%')), y=alt.Y("advantage:N",sort=[]))
    .transform_filter("datum.age=='Overall'")
    .transform_calculate(x0="0")
)
legendkey='Fewer distractions'
legendx=0.7
legendfs=11
legend1 = (
    bars1.mark_text(align="right", size=legendfs, yOffset=-14, color=colors["eco-turquiose"])
    .encode(text="age:N", x="x0:Q")
    .transform_filter("datum.advantage=='"+legendkey+"'")
    .transform_calculate(x0=str(legendx))
)
legend2 = (
    bars2.mark_text(align="right", size=legendfs, yOffset=-27, color=colors["eco-green"])
    .encode(text="age:N", x="x0:Q")
    .transform_filter("datum.advantage=='"+legendkey+"'")
    .transform_calculate(x0=str(legendx))
)
legend3 = (
    bars3.mark_text(align="right", size=legendfs, yOffset=0, color=colors["eco-light-blue"])
    .encode(text="age:N", x="x0:Q")
    .transform_filter("datum.advantage=='"+legendkey+"'")
    .transform_calculate(x0=str(legendx))
)
legend4 = (
    bars4.mark_text(align="right", size=legendfs, yOffset=13, color=colors["eco-mid-blue"])
    .encode(text="age:N", x="x0:Q")
    .transform_filter("datum.advantage=='"+legendkey+"'")
    .transform_calculate(x0=str(legendx))
)

layer = (
    (
        (bars1 + legend1)
        + (bars2 + legend2)
        + (bars3 + legend3)
        + (bars4 + legend4)
        + labels1
        + labels2
        + labels3
        + labels4
        + titles
    )
    .configure_view(stroke=None)
    .properties(title="")
    .properties(height=alt.Step(75), width=400)
)
layer.save("visualisation/" + f + ".json")
layer.save("visualisation/" + f + ".png")
open("README.md",'a').write(readme)
layer

Fontconfig error: Cannot load default config file


alt.LayerChart(...)

# Fig 3

In [113]:
df = pd.read_excel("raw/figures.xlsx",sheet_name='Fig2_Fig3',skiprows=15,usecols='B:F',nrows=10)
df=df.set_index(df.columns[0]).stack().reset_index()
df.columns=['advantage','age','value']
df['value']/=100
df=df[::-1]

In [114]:
f = "fig3_disadvantages"
f3 = eco_git_path + f + ".csv"
df.to_csv("data/" + f + ".csv")
f += local_suffix
open("visualisation/" + f + ".html", "w").write(
    vega_embed.replace(
        "JSON_PATH", f3.replace("/data/", "/visualisation/").replace(".csv", ".json")
    )
)
if LOCAL:
    f3 = df
readme="### "+f+'\n!["'+f+'"](visualisation/'+f+'.png "'+f+'")\n\n'
df.head()

,advantage,age,value
39,Harder to work with others,50-69,0.36
38,Harder to work with others,30-49,0.45
37,Harder to work with others,16-29,0.56
36,Harder to work with others,Overall,0.45
35,More distractions,50-69,0.21


In [119]:
bars = alt.Chart(f3).encode(
    x=alt.X(
        "value:Q",
        stack=False,
        title="",
        axis=alt.Axis(
            grid=False,
            title="",
            titleAnchor="end",
            titleX=415,
            titleY=7,
            format='.0%',
            labelColor=colors["eco-gray"],
            titleColor=colors["eco-gray"],
            tickColor=colors["eco-gray"],
            domainColor=colors["eco-gray"],
        ),
    ),
    y=alt.Y("advantage:N", title="", axis=None, sort=[]),
    tooltip="tooltip:N"
).transform_calculate(tooltip="datum.advantage+' 🏢 '+datum.age+' 📈 '+round(datum.value*100)+'%'")

bars1 = bars.mark_bar(
    size=11, yOffset=-14, color=colors["eco-turquiose"], opacity=0.8
).transform_filter("datum.age=='16-29'")
bars2 = bars.mark_bar(
    size=11, yOffset=-26, color=colors["eco-green"], opacity=0.8
).transform_filter("datum.age=='Overall'")
bars3 = bars.mark_bar(
    size=11, yOffset=-2, color=colors["eco-light-blue"], opacity=0.8
).transform_filter("datum.age=='30-49'")
bars4 = bars.mark_bar(
    size=11, yOffset=10, color=colors["eco-mid-blue"], opacity=0.8
).transform_filter("datum.age=='50-69'")
labels1 = bars1.mark_text(
    align="left",
    xOffset=5,
    yOffset=-14,
    color=colors["eco-turquiose"],
).encode(text=alt.Text("value:N",format='.0%'))
labels2 = bars2.mark_text(
    align="left",
    xOffset=5,
    yOffset=-26,
    color=colors["eco-green"],
).encode(text=alt.Text("value:N",format='.0%'))
labels3 = bars3.mark_text(
    align="left",
    xOffset=5,
    yOffset=-2,
    color=colors["eco-light-blue"],
).encode(text=alt.Text("value:N",format='.0%'))
labels4 = bars4.mark_text(
    align="left",
    xOffset=5,
    yOffset=10,
    color=colors["eco-mid-blue"],
).encode(text=alt.Text("value:N",format='.0%'))
titles = (
    bars.mark_text(
        align="left", size=11, baseline="bottom", yOffset=-35, color=colors["eco-gray"]
    )
    .encode(text="advantage:N", x=alt.X("x0:Q",axis=alt.Axis(format='.0%')), y=alt.Y("advantage:N",sort=[]))
    .transform_filter("datum.age=='Overall'")
    .transform_calculate(x0="0")
)
legendkey='More distractions'
legendx=0.58
legendfs=11
legend1 = (
    bars1.mark_text(align="right", size=legendfs, yOffset=-14, color=colors["eco-turquiose"])
    .encode(text="age:N", x="x0:Q")
    .transform_filter("datum.advantage=='"+legendkey+"'")
    .transform_calculate(x0=str(legendx))
)
legend2 = (
    bars2.mark_text(align="right", size=legendfs, yOffset=-27, color=colors["eco-green"])
    .encode(text="age:N", x="x0:Q")
    .transform_filter("datum.advantage=='"+legendkey+"'")
    .transform_calculate(x0=str(legendx))
)
legend3 = (
    bars3.mark_text(align="right", size=legendfs, yOffset=0, color=colors["eco-light-blue"])
    .encode(text="age:N", x="x0:Q")
    .transform_filter("datum.advantage=='"+legendkey+"'")
    .transform_calculate(x0=str(legendx))
)
legend4 = (
    bars4.mark_text(align="right", size=legendfs, yOffset=13, color=colors["eco-mid-blue"])
    .encode(text="age:N", x="x0:Q")
    .transform_filter("datum.advantage=='"+legendkey+"'")
    .transform_calculate(x0=str(legendx))
)

layer = (
    (
        (bars1 + legend1)
        + (bars2 + legend2)
        + (bars3 + legend3)
        + (bars4 + legend4)
        + labels1
        + labels2
        + labels3
        + labels4
        + titles
    )
    .configure_view(stroke=None)
    .properties(title="")
    .properties(height=alt.Step(75), width=400)
)
layer.save("visualisation/" + f + ".json")
layer.save("visualisation/" + f + ".png")
open("README.md",'a').write(readme)
layer

Fontconfig error: Cannot load default config file


alt.LayerChart(...)